In [ ]:
# !pip install pyts

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from pyts.image import MarkovTransitionField

import scipy
from scipy.io import loadmat

from pywt import wavedec
from sklearn.model_selection import KFold

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold,cross_validate
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LogisticRegression 
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score, classification_report, accuracy_score
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
import sklearn.svm as svm 
import sklearn.naive_bayes as naive_bayes


import tensorflow.compat.v1 as tf
from tensorflow import keras as K
from tensorflow.keras.applications.resnet50 import ResNet50

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Dense, Activation, Flatten, concatenate, Input, Dropout,Conv1D, LSTM, Bidirectional,BatchNormalization,PReLU,ReLU,Reshape
from tensorflow.keras.layers import Conv1D,Conv2D,Add
from tensorflow.keras.layers import MaxPool1D, MaxPooling2D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow import keras
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils.np_utils import to_categorical

import seaborn as sns

import xgboost as xgb

In [ ]:
subject = 3

In [ ]:
input_data = pd.read_csv("/kaggle/input/time-series-data/Final_inputs_"+str(subject)+".csv")
output_data = pd.read_csv("/kaggle/input/time-series-data/Final_outputs_"+str(subject)+".csv")

In [ ]:
input_data

In [ ]:
output_data

In [ ]:
temp_coco = input_data.iloc[3].tolist()
temp_image = input_data.iloc[4].tolist()
temp_sun = input_data.iloc[5].tolist()

In [ ]:
def clean(lst):
    new_lst = []
    for i in lst:
        if i != 0:
            new_lst.append(i)
            
    return new_lst
            
fin_coco = clean(temp_coco)
fin_image = clean(temp_image)
fin_sun = clean(temp_sun)

In [ ]:
len(fin_coco) + len(fin_image) + len(fin_sun)

In [ ]:
from scipy.interpolate import make_interp_spline

def plot(lst,string):
    
    x = [i for i in range(len(lst))]
    X_Y_Spline = make_interp_spline(x, lst)
    X_ = np.linspace(min(x), max(x), 500)
    Y_ = X_Y_Spline(X_)

    # Plotting the Graph
    plt.plot(X_, Y_)
    plt.title(string)
    plt.xlabel("Time")
    plt.ylabel("Bold Intensity")
    return plt


plt = plot(fin_coco,"Time series of COCO")

In [ ]:
plt = plot(fin_image,"Time series of ImageNet")

In [ ]:
plt = plot(fin_sun,"Time series of SUN")

In [ ]:
x = input_data.to_numpy()
y = output_data.to_numpy()

In [ ]:
x

In [ ]:
y

In [ ]:
x = StandardScaler().fit_transform(x)

In [ ]:
x

In [ ]:
len(y[0])

In [ ]:
# x_train.shape

In [ ]:
mtf = MarkovTransitionField(image_size=37)

In [ ]:
x = mtf.fit_transform(x)
# im_test = mtf.transform(x_test)

In [ ]:
plt.imshow(x[3])

In [ ]:
plt.imshow(x[4])

In [ ]:
plt.imshow(x[5])

In [ ]:
# im_train.shape

In [ ]:
# im_test.shape

In [ ]:
x.shape

In [ ]:
x = x.reshape(x.shape[0], 37, 37,1)
# im_test = im_test.reshape(im_test.shape[0], 37, 37,1)

In [ ]:
x.shape

In [ ]:
im_train,im_test,y_train,y_test = train_test_split(x,y,test_size=0.25)

In [ ]:
# im_test.shape

In [ ]:
# x_train[0]

In [ ]:
# im_train[0]

In [ ]:
# ly_train = label_binarize(y_train, classes=[0, 1, 2])
# ly_test = label_binarize(y_test, classes=[0, 1, 2])

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(37, (3, 3), activation='relu', input_shape=(37, 37,1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3))

model.summary()

In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
def train_model(model,x_train, y_train,x_test,y_test, save_to, epoch = 2):

        opt_adam = keras.optimizers.Adam(learning_rate=0.001)

        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
        mc = ModelCheckpoint(save_to + '_best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
        lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 0.001 * np.exp(-epoch / 10.))
        
        model.compile(optimizer=opt_adam,loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
        
        history = model.fit(x_train,y_train,
                        batch_size=20,
                        epochs=epoch,
                        validation_data=(x_test,y_test),
                        callbacks=[es,mc,lr_schedule])
        
        model = load_model(save_to + '_best_model.h5')
        
        return model,history

In [ ]:
# model,history = train_model(model, im_train, y_train,im_test,y_test, save_to= './', epoch = 100) 

In [ ]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

acc_per_fold = []
loss_per_fold = []
true_labels = []
pred_labels = []
pred_probs = []

fold_no = 1

In [ ]:
# model = load_model('./_best_model.h5')
for train, test in kfold.split(x,y): 
    model = models.Sequential()
    model.add(layers.Conv2D(37, (3, 3), activation='relu', input_shape=(37, 37,1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(3))
    
    model,history = train_model(model, x[train], y[train], x[test], y[test], save_to= './', epoch = 50) 


    scores = model.evaluate(x[test], y[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    #   print("***************************************************\n")
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    y_pred = model.predict(x[test])
    pred_probs.extend(y_pred)
    fin_lst = []
    for i in y_pred:
        fin_lst.append(np.argmax(i, axis=0))

    fin_lst = np.array(fin_lst)
    fin_lst = fin_lst.reshape((-1,1))
#     y_hat = np.array(y_pred >= 0.5, dtype = int)
    true_labels.extend(y[test])
    pred_labels.extend(fin_lst)

    # Increase fold number
    fold_no = fold_no + 1

In [ ]:
print(acc_per_fold)

In [ ]:
acc = np.mean(acc_per_fold)
acc ="{0:0.2f}".format(acc*0.01)
acc

In [ ]:
print(classification_report(true_labels, pred_labels,labels=[0,1,2]))

In [ ]:
precision = classification_report(true_labels, pred_labels,labels=[0,1,2])[291:295]
precision

In [ ]:
recall = classification_report(true_labels, pred_labels,labels=[0,1,2])[301:305]
recall

In [ ]:
f1 = classification_report(true_labels, pred_labels,labels=[0,1,2])[311:315]
f1

In [ ]:
ly_test = label_binarize(true_labels, classes=[0, 1, 2])
test_yhat = label_binarize(pred_labels, classes=[0, 1, 2])

In [ ]:
n_classes = 3
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(ly_test[:, i], test_yhat[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(ly_test.ravel(), test_yhat.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
# plt.plot(
#     fpr["micro"],
#     tpr["micro"],
#     label="micro-average ROC curve (area = {0:0.2f})".format(roc_auc["micro"]),
#     color="deeppink",
#     linewidth=4,
# )

plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="(area = {0:0.2f})".format(roc_auc["macro"]),
    color="red",
#     linestyle=":",
    linewidth=4,
)

# colors = cycle(["aqua", "darkorange", "cornflowerblue"])
# for i, color in zip(range(n_classes), colors):
#     plt.plot(
#         fpr[i],
#         tpr[i],
#         color=color,
#         lw=2,
#         label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
#     )

plt.plot([0, 1], [0, 1], "k--", lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
# plt.title("Some extension of Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.savefig('./roc_'+str(subject)+'.png')
plt.show()

In [ ]:
roc="{0:0.2f}".format(roc_auc["macro"])
roc